Paquetes

In [271]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random

import optuna
import tqdm as notebook_tqdm

Carga de datos

In [70]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## **Consigna del desafío 1**

### **Parte 1**. 
Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.


In [193]:
# Instanciamos un vectorizador
tfidfvect = TfidfVectorizer()

# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)

# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target

In [194]:
print(type(X_train))
print(f'Shape: {X_train.shape}')
print(f'Number of documents: {X_train.shape[0]}')
print(f'Vocabulary size: {X_train.shape[1]}')

# para ver los grupos de noticias
print(newsgroups_test.target_names)

<class 'scipy.sparse._csr.csr_matrix'>
Shape: (11314, 101631)
Number of documents: 11314
Vocabulary size: 101631
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [196]:
np.random.seed(41)
np.random.choice(X_train.shape[0], 5)

array([1984,  931, 4066, 5200, 8513])

##### Revisión Documento: 1984

Primero se revisa que tiene el documento de referencia.

In [197]:
revision_idx = 1984

print("--------------------------------------------------")
print(f'CLASE DEL DOCUMENTO: {newsgroups_train.target_names[y_train[revision_idx]]}')
print("--------------------------------------------------")
print(newsgroups_train.data[revision_idx])


--------------------------------------------------
CLASE DEL DOCUMENTO: misc.forsale
--------------------------------------------------
Panasonic KX-T3000H, Combo black cordless & speaker phone all in one.
 new- $160, now- $100 + shipping OBO.
 
Curtis Mathes VHS VCR Remote included and it works with universal remotes.
 Works great but I replaced it with a Stereo VCR.
 paid $300 years ago, will sell for $125 delivered OBO.
 
Radio Shack stereo amp.  2 inputs, tone, and left and right volume.  Speakers
 not included.  $20 plus shipping.
 
If you are interested in either of the above mail me at
 radley@gibbs.oit.unc.edu or call me, Keith, at 919-968-7779.
 
PS- I made a type on my email address the first posting.  It is now correct.


Este documento hace referencia a venta de productos electrónicos. En este caso en particular, parece una oferta enviada a un cliente. Veamos con que documentos tiene menor distancia coseno (y por lo tanto, mayor similaridad).

In [198]:
# Medimos la distancia coseno con los demás documentos disponibles
cos_sim = cosine_similarity(X_train[revision_idx], X_train)[0]

# Devolvemos los primeros 5 (sin contar el documento de revisión)
documentos_similares = np.argsort(cos_sim)[::-1][1:6]
print(documentos_similares)

[2028 2878 3794 4644 8109]


In [199]:
for i in documentos_similares:
    
    print("--------------------------------------------------")
    print(f'CLASE DEL DOCUMENTO {i}: {newsgroups_train.target_names[y_train[i]]}')
    print("--------------------------------------------------")
    print(newsgroups_train.data[i])
    print("\n")

--------------------------------------------------
CLASE DEL DOCUMENTO 2028: misc.forsale
--------------------------------------------------
Panasonic KX-T3000H, Combo black cordless & speaker phone all in one.
 new- $160, now- $100 + shipping OBO.
 
Curtis Mathes VHS VCR Remote included and it works with universal remotes.
 Works great but I replaced it with a Stereo VCR.
 paid $300 years ago, will sell for $125 delivered OBO.
 
Radio Shack stereo amp.  2 inputs, tone, and left and right volume.  Speakers
 not included.  $20 plus shipping.
 
If you are interested in either of the above mail me at
 radley@gibbs.out.unc.edu.


--------------------------------------------------
CLASE DEL DOCUMENTO 2878: misc.forsale
--------------------------------------------------
Panasonic KX-T3000H, Combo black cordless & speaker phone all in one.
 new- $160, now- $100 + shipping OBO.
 
Curtis Mathes VHS VCR Remote included and it works with universal remotes.
 Works great but I replaced it with a St

En este caso, todos los documentos se encuentran a la clase `misc.forsale`, por lo que se entiende son ofertas o promociones de artículos electrónicos, realizadas a clientes. Varios de ellos tienen una estructura similar, describiendo el producto, agregando precio del mismo y difieren en los datos de contacto. Por lo que son documentos muy similiar en cuanto a lo cualitativo.

##### Revisión Documento: 931



Primero se revisa que tiene el documento

In [200]:
revision_idx = 931

print("--------------------------------------------------")
print(f'CLASE DEL DOCUMENTO: {newsgroups_train.target_names[y_train[revision_idx]]}')
print("--------------------------------------------------")
print(newsgroups_train.data[revision_idx])


--------------------------------------------------
CLASE DEL DOCUMENTO: sci.crypt
--------------------------------------------------
I have lurked here a bit lately, and though some of the math is
unknown to me, found it interesting. I thought I would post an article I
found in the Saturday, April 17, 1993 Toronto Star:

                  'CLIPPER CHIP' to protect privacy

   Washington (REUTER) - President Bill CLinton announced yesterday a
plan to plant a new "Clipper Chip" in every government telephone and
computer line to prevent eavesdropping.

   Eventually the chips, developed by the government's National
Institute for Standards and Technology, would be used by commercial and
private electronics communication users.

   The White House said that to assure privacy, each device containing
the encryption devices would be assigned two unique "keys" - numbers
that will be needed by government agencies to decode messages.

   The attorney-general has been assigned the task of arrangin

In [201]:
# Medimos la distancia coseno con los demás documentos disponibles
cos_sim = cosine_similarity(X_train[revision_idx], X_train)[0]

# Devolvemos los primeros 5 (sin contar el documento de revisión)
documentos_similares = np.argsort(cos_sim)[::-1][1:6]
print(documentos_similares)

[9396 8445 9007 5612 4498]


In [202]:
for i in documentos_similares:
    
    print("--------------------------------------------------")
    print(f'CLASE DEL DOCUMENTO {i}: {newsgroups_train.target_names[y_train[i]]}')
    print("--------------------------------------------------")
    print(newsgroups_train.data[i])
    print("\n")

--------------------------------------------------
CLASE DEL DOCUMENTO 9396: sci.crypt
--------------------------------------------------


Here is an article I found today in comp.security.misc. I'll send my reply in
a separate post to comp.off.eff.org so thayt you guys can get original text.
Have fun! ;(

Oleg

   Relay-Version: VMS News - V6.1B5 17/9/92 VAX/VMS V5.5-2; site nic.csu.net
   Path: nic.csu.net!csus.edu!netcom.com!netcomsv!decwrl!uunet!dove!csrc.ncsl.nist.gov!clipper
   Newsgroups: comp.security.misc
   From: clipper@csrc.ncsl.nist.gov (Clipper Chip Announcement)
   Date: Fri, 16 Apr 1993 15:17:33 GMT
   Sender: news@dove.nist.gov
   Distribution: na
   Organization: National Institute of Standards & Technology
   Lines: 280

   Note:  This file will also be available via anonymous file
   transfer from csrc.ncsl.nist.gov in directory /pub/nistnews and
   via the NIST Computer Security BBS at 301-948-5717.
	---------------------------------------------------

			    THE 

En este caso, todos los documentos se encuentran a la clase `sci.scrypt`, haciendo referencia a un modo de encriptación llamada `Clipper Chip` (parece ser un tipo de llave digital). En cuanto a la similaridad de contenido, todos estos documentos comparten detalles específicos sobre el propósito del "Clipper Chip" para la protección de comunicaciones y las medidas de seguridad para el acceso a las claves de cifrado por parte de agencias gubernamentales con autorización legal

##### Revisión Documento: 4066

In [203]:
revision_idx = 4066

print("--------------------------------------------------")
print(f'CLASE DEL DOCUMENTO: {newsgroups_train.target_names[y_train[revision_idx]]}')
print("--------------------------------------------------")
print(newsgroups_train.data[revision_idx])


--------------------------------------------------
CLASE DEL DOCUMENTO: talk.politics.misc
--------------------------------------------------
Go easy on him drieux. It is the right of every American to
know nothing about anything. 



------------------------------------------------------------------------------
"Who said anything about panicking?" snapped Authur.           Garrett Johnson
"This is still just culture shock. You wait till I've       Garrett@Ingres.com
settled into the situation and found my bearings.
THEN I'll start panicking!" - Douglas Adams  


In [204]:
# Medimos la distancia coseno con los demás documentos disponibles
cos_sim = cosine_similarity(X_train[revision_idx], X_train)[0]

# Devolvemos los primeros 5 (sin contar el documento de revisión)
documentos_similares = np.argsort(cos_sim)[::-1][1:6]
print(documentos_similares)

[3724 9348 3384 3739 8667]


In [205]:
for i in documentos_similares:
    
    print("--------------------------------------------------")
    print(f'CLASE DEL DOCUMENTO {i}: {newsgroups_train.target_names[y_train[i]]}')
    print("--------------------------------------------------")
    print(newsgroups_train.data[i])
    print("\n")

--------------------------------------------------
CLASE DEL DOCUMENTO 3724: talk.politics.misc
--------------------------------------------------
Just how much power does the House of Lords have now? 

------------------------------------------------------------------------------
"Who said anything about panicking?" snapped Authur.           Garrett Johnson
"This is still just culture shock. You wait till I've       Garrett@Ingres.com
settled into the situation and found my bearings.
THEN I'll start panicking!" - Douglas Adams  


--------------------------------------------------
CLASE DEL DOCUMENTO 9348: talk.politics.misc
--------------------------------------------------

I guess your strength isn't in math. Clinton hasn't been president for
6 months. In other words, it's BUSH'S Wiretapping Initiative.
Have you?

------------------------------------------------------------------------------
"Who said anything about panicking?" snapped Authur.           Garrett Johnson
"This is sti

En este caso, todos los documentos se encuentran a la clase `talk.politics.misc`, presentando todos ellos similitudes en contenido y estilo. Esto último puede deberse, a que, quien los escribe, parece ser la misma persona. Al final de cada documento, aparece la misma cita de *Douglas Adams*, por lo que es esperable que los tome como documentos similares.

##### Revisión Documento: 5200



Primero se revisa que tiene el documento

In [206]:
revision_idx = 5200

print("--------------------------------------------------")
print(f'CLASE DEL DOCUMENTO: {newsgroups_train.target_names[y_train[revision_idx]]}')
print("--------------------------------------------------")
print(newsgroups_train.data[revision_idx])


--------------------------------------------------
CLASE DEL DOCUMENTO: alt.atheism
--------------------------------------------------
Archive-name: atheism/introduction
Alt-atheism-archive-name: introduction
Last-modified: 5 April 1993
Version: 1.2

-----BEGIN PGP SIGNED MESSAGE-----

                          An Introduction to Atheism
                       by mathew <mathew@mantis.co.uk>

This article attempts to provide a general introduction to atheism.  Whilst I
have tried to be as neutral as possible regarding contentious issues, you
should always remember that this document represents only one viewpoint.  I
would encourage you to read widely and draw your own conclusions; some
relevant books are listed in a companion article.

To provide a sense of cohesion and progression, I have presented this article
as an imaginary conversation between an atheist and a theist.  All the
questions asked by the imaginary theist are questions which have been cropped
up repeatedly on alt.atheis

Este documento pertenece a la clase `alt.atheism` y trata sobre temas religiosos, específicamente sobre el ateísmo. Parece tener un formato de entrevista, con preguntas y respuestas brindadas. Revisemos los documentos más similares a este según la distancia coseno.

In [207]:
# Se mide la distancia coseno con los demás documentos disponibles
cos_sim = cosine_similarity(X_train[revision_idx], X_train)[0]

# Devolvemos los primeros 5 (sin contar el documento de revisión)
documentos_similares = np.argsort(cos_sim)[::-1][1:6]
print(documentos_similares)

[10836  2095  7539 10924  8795]


In [208]:
for i in documentos_similares:
    
    print("--------------------------------------------------")
    print(f'CLASE DEL DOCUMENTO {i}: {newsgroups_train.target_names[y_train[i]]}')
    print("--------------------------------------------------")
    print(newsgroups_train.data[i])
    print("\n")

--------------------------------------------------
CLASE DEL DOCUMENTO 10836: alt.atheism
--------------------------------------------------
Archive-name: atheism/faq
Alt-atheism-archive-name: faq
Last-modified: 5 April 1993
Version: 1.1

                    Alt.Atheism Frequently-Asked Questions

This file contains responses to articles which occur repeatedly in
alt.atheism.  Points covered here are ones which are not covered in the
"Introduction to Atheism"; you are advised to read that article as well
before posting.

These answers are not intended to be exhaustive or definitive. The purpose of
the periodic FAQ postings is not to stifle debate, but to raise its level. If
you have something to say concerning one of these questions and which isn't
covered by the answer given, please feel free to make your point.

Overview of contents:

   "What is the purpose of this newsgroup?"
   "Hitler was an atheist!"
   "The Bible proves it"
   "Pascal's Wager"
   "What is Occam's Razor?"
   "Wh

Casi todos los documentos pertenecen a la misma categoría, `alt.atheism`, excepto por el `8795` que pertenece a `soc.religion.christian`, aunque todos se centran en temas relacionados con el ateísmo y el escepticismo respecto a creencias religiosas.

##### Revisión Documento: 8513

In [209]:
revision_idx = 8513

print("--------------------------------------------------")
print(f'CLASE DEL DOCUMENTO: {newsgroups_train.target_names[y_train[revision_idx]]}')
print("--------------------------------------------------")
print(newsgroups_train.data[revision_idx])


--------------------------------------------------
CLASE DEL DOCUMENTO: comp.os.ms-windows.misc
--------------------------------------------------
Hey now.  First of all, sorry to post this DOS question in a WINDOWS
group, but I'm in kinda a hurry, so I can't scramble to find the dos
groups' names.  

Anyway, anyone know where I ccan find the exit codes to DOS commands?
the manual doesn't seem to have all of them.  I'm particularly looking
for COPY, in order to make a "move" batch file, such that if the file
wasn't coppied properly, it won't be deleted.


please e'mail louray@seas.gwu.edu
Thanks, I.A,
Mickey


Este documento pertenece a la clase `comp.os.ms-windows.misc` y parece hablar sobre temas técnicos, específicamente una consulta en un grupo de discusión de Windows. Veremos que otros documentos guardan similaridad con este.

In [210]:
# Se mide la distancia coseno con los demás documentos disponibles
cos_sim = cosine_similarity(X_train[revision_idx], X_train)[0]

# Primeros 5 (sin contar el documento de revisión)
documentos_similares = np.argsort(cos_sim)[::-1][1:6]
print(documentos_similares)

[5160 5015 4255 5977 2140]


In [211]:
for i in documentos_similares:
    
    print("--------------------------------------------------")
    print(f'CLASE DEL DOCUMENTO {i}: {newsgroups_train.target_names[y_train[i]]}')
    print("--------------------------------------------------")
    print(newsgroups_train.data[i])
    print("\n")

--------------------------------------------------
CLASE DEL DOCUMENTO 5160: comp.os.ms-windows.misc
--------------------------------------------------
I'm looking for a program that will let me use a windows
common dialog box to select a file to use when running a DOS app.

	Basically, I have several DOS apps that I use now and then
with different files.  They all accept a file as a command line 
parameter, but the only way (at least that I know of) to do this
easily when running them from windows is to set up the PIF file so
that it prompts me for additional parameters (at which point I type 
in the file name).  

Problems are:

	1) Sometimes I can't remember where the file is exactly located and it
would be nice to browse my directories for it without having to use the
file manager.
	
    and 2) I'm lazy and hate to type long pathnames for files burried 
several directory levels deep.

	If anyone can point me to such a program or let me know of some
other way to handle this, I'd app

Todas parecen ser consultas técnicas, pricnipalmente en foros, sobre el sistema operativos Windows y el uso de DOS. En este caso no comparten la misma clase los documentos, aunque si guardan similaridad en el contenido.

### **Parte 2**

Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámetros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.


Modelo Naives Bayes Multinomial **sin ajuste de hiperparámetros** 

In [255]:
# Modelo de clasificación Naïve Bayes Multinomial y entrenamiento
clf_MNB = MultinomialNB()
clf_MNB.fit(X_train, y_train)

# Vectorizamos los textos del conjunto de test
X_test_MNB = tfidfvect.transform(newsgroups_test.data)
y_test_MNB = newsgroups_test.target
y_pred_MNB =  clf_MNB.predict(X_test_MNB)

print(f'F1-Score Naive Bayes Multinomial: {f1_score(y_test_MNB, y_pred_MNB, average='macro')}')

F1-Score Naive Bayes Multinomial: 0.6564514103512165


Ahora se realiza búsqueda de hiperparámetros con Optuna para este modelo y se obtiene el nuevo modelo **con ajuste de hiperparámetros**.

In [263]:
def objective(trial):
    # Rango de valores para los hiperparámetros de TfidfVectorizer
    max_df = trial.suggest_float('max_df', 0.5, 1.0)
    min_df = trial.suggest_int('min_df', 1, 10)
    ngram_range_str = trial.suggest_categorical('ngram_range', ['(1, 1)', '(1, 2)', '(1, 3)'])
    ngram_range = eval(ngram_range_str)  # Convierte el string en tupla
    
    # TfidfVectorizer con los hiperparámetros sugeridos
    tfidfvect = TfidfVectorizer(max_df=max_df, min_df=min_df, ngram_range=ngram_range)
    X_train = tfidfvect.fit_transform(newsgroups_train.data)
    y_train = newsgroups_train.target
    X_test = tfidfvect.transform(newsgroups_test.data)
    y_test = newsgroups_test.target

    # Valores para el hiperparámetro alpha de MultinomialNB
    alpha = trial.suggest_float('alpha', 1e-3, 1.0, log=True)
    
    # MultinomialNB con el valor de alpha sugerido
    clf_MNB = MultinomialNB(alpha=alpha)
    clf_MNB.fit(X_train, y_train)
    
    # Predicción y cálculo de F1-Score
    y_pred = clf_MNB.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    
    return f1

# Ejecución y búsqueda con Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Mejores hiperparámetros encontrados
print(f"\n")
print(f"Mejores parámetros: {study.best_params}")
print(f"Mejor F1-Score: {study.best_value}")

[I 2024-10-31 16:28:16,708] A new study created in memory with name: no-name-b8341047-4f6e-4c2e-87ba-ebbdee763ad6
[I 2024-10-31 16:28:20,623] Trial 0 finished with value: 0.6564420440129047 and parameters: {'max_df': 0.6553609682561681, 'min_df': 9, 'ngram_range': '(1, 1)', 'alpha': 0.2044828100984398}. Best is trial 0 with value: 0.6564420440129047.
[I 2024-10-31 16:28:39,436] Trial 1 finished with value: 0.623273793319699 and parameters: {'max_df': 0.94211571307529, 'min_df': 5, 'ngram_range': '(1, 3)', 'alpha': 0.005031524893916752}. Best is trial 0 with value: 0.6564420440129047.
[I 2024-10-31 16:28:55,266] Trial 2 finished with value: 0.6285124962321199 and parameters: {'max_df': 0.508888601296628, 'min_df': 9, 'ngram_range': '(1, 3)', 'alpha': 0.13886538476223684}. Best is trial 0 with value: 0.6564420440129047.
[I 2024-10-31 16:28:58,934] Trial 3 finished with value: 0.6738190938154149 and parameters: {'max_df': 0.6569757578285699, 'min_df': 6, 'ngram_range': '(1, 1)', 'alpha': 



Mejores parámetros: {'max_df': 0.6569757578285699, 'min_df': 6, 'ngram_range': '(1, 1)', 'alpha': 0.0434500473671063}
Mejor F1-Score: 0.6738190938154149


Con el ajuste de hiperparámetros, el F1-score aumenta en un 2.6% aprox. por lo que tiene efecto la optimización.

Complement Naives Bayes Multinomial **sin ajuste de hiperparámetros** 

In [266]:
# Modelo de clasificación ComplementNB y entrenamiento
clf_CNB = ComplementNB(alpha=0.95)
clf_CNB.fit(X_train, y_train)

# Vectorizamos los textos del conjunto de test
X_test_CNB = tfidfvect.transform(newsgroups_test.data)
y_test_CNB = newsgroups_test.target
y_pred_CNB =  clf_CNB.predict(X_test_CNB)

print(f'F1-Score Complement Naive Bayes: {f1_score(y_test_CNB, y_pred_CNB, average='macro')}')

F1-Score Complement Naive Bayes: 0.6924226432787461


Ahora se realiza búsqueda de hiperparámetros con Optuna para este modelo y se obtiene el nuevo modelo **con ajuste de hiperparámetros**.

In [268]:
def objective(trial):
    # Rango de valores para los hiperparámetros de TfidfVectorizer
    max_df = trial.suggest_float('max_df', 0.5, 1.0)
    min_df = trial.suggest_int('min_df', 1, 10)
    ngram_range_str = trial.suggest_categorical('ngram_range', ['(1, 1)', '(1, 2)', '(1, 3)'])
    ngram_range = eval(ngram_range_str)  # Convierte el string en tupla
    
    # TfidfVectorizer con los hiperparámetros sugeridos
    tfidfvect = TfidfVectorizer(max_df=max_df, min_df=min_df, ngram_range=ngram_range)
    X_train = tfidfvect.fit_transform(newsgroups_train.data)
    y_train = newsgroups_train.target
    X_test = tfidfvect.transform(newsgroups_test.data)
    y_test = newsgroups_test.target

    # Valores para el hiperparámetro alpha de MultinomialNB
    alpha = trial.suggest_float('alpha', 1e-3, 1.0, log=True)
    
    # MultinomialNB con el valor de alpha sugerido
    clf_CNB = ComplementNB(alpha=alpha)
    clf_CNB.fit(X_train, y_train)
    
    # Predicción y cálculo de F1-Score
    y_pred = clf_CNB.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    
    return f1

# Ejecución y búsqueda con Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Mejores hiperparámetros encontrados
print(f"\n")
print(f"Mejores parámetros: {study.best_params}")
print(f"Mejor F1-Score: {study.best_value}")

[I 2024-10-31 16:32:15,130] A new study created in memory with name: no-name-ef024544-cf1f-4852-a8a6-3314da548101
[I 2024-10-31 16:32:19,060] Trial 0 finished with value: 0.6695680210212924 and parameters: {'max_df': 0.534427254306985, 'min_df': 5, 'ngram_range': '(1, 1)', 'alpha': 0.005561772374514494}. Best is trial 0 with value: 0.6695680210212924.
[I 2024-10-31 16:32:50,793] Trial 1 finished with value: 0.6779060872239857 and parameters: {'max_df': 0.5296405515604743, 'min_df': 1, 'ngram_range': '(1, 3)', 'alpha': 0.6038994237988009}. Best is trial 1 with value: 0.6779060872239857.
[I 2024-10-31 16:33:07,082] Trial 2 finished with value: 0.7065847716484188 and parameters: {'max_df': 0.7947876741038434, 'min_df': 1, 'ngram_range': '(1, 2)', 'alpha': 0.12383158611134679}. Best is trial 2 with value: 0.7065847716484188.
[I 2024-10-31 16:33:10,794] Trial 3 finished with value: 0.666142100360257 and parameters: {'max_df': 0.7007132008217494, 'min_df': 8, 'ngram_range': '(1, 1)', 'alpha'



Mejores parámetros: {'max_df': 0.7947876741038434, 'min_df': 1, 'ngram_range': '(1, 2)', 'alpha': 0.12383158611134679}
Mejor F1-Score: 0.7065847716484188


Con el ajuste de hiperparámetros, el F1-score aumenta en un 2% aprox. por lo que tiene efecto la optimización. Quizás con más pruebas se podría mejorar un poco más.

### **Parte 3**

Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

Se instancia el vectorizador con TF-IDF para generar la matriz termino-documento y se transpone.

In [355]:
vectorizer = TfidfVectorizer( 
    stop_words='english', 
    max_features=10000, 
    sublinear_tf=True, 
    ngram_range=(1, 1), 
    min_df=51, 
    max_df=0.7947876741038434)
X_train_dt = vectorizer.fit_transform(newsgroups_train.data)

# Se transpone
X_docs = X_train_dt.T
terms = vectorizer.get_feature_names_out()

In [391]:
# Diccionario que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [495]:
# Palabras elegidas (manualmente) para buscar términos similares
palabras_target = ['history','graphics','games','market','data'] 

Revisión palabra objetivo: **'history'**

In [496]:
# Índice de la palabra en `terms` fila correspondiente en la matriz de documentos
idx_palabra = terms.tolist().index(palabras_target[0])
vector_palabra = X_docs[idx_palabra].toarray()
cos_term_sim = cosine_similarity(vector_palabra, X_docs)

# Se traen los índices de las palabras más similares de acuerdo al orden de distancia coseno
idx_similares = np.argsort(cos_term_sim[0])[::-1][1:6]
print(f"Indices de palabras similares: {idx_similares} \n")
print([(terms[i], np.round(cos_term_sim[0][i],4)) for i in idx_similares])

Indices de palabras similares: [2489 1068 2589  258 2585] 

[('turks', 0.1456), ('genocide', 0.1417), ('wars', 0.1295), ('armenians', 0.1144), ('war', 0.1102)]


En este caso las palabras más similares, guardan cierta relación con la palabra objetivo `History`, haciendo énfasis en conflictos bélicos (`war`,`wars`,`genocide`), particularmente entre pueblos con historia conocida (`armenians`,`turks`).

Revisión palabra objetivo: **'graphics'**

In [497]:
# Índice de la palabra en `terms` fila correspondiente en la matriz de documentos
idx_palabra = terms.tolist().index(palabras_target[1])
vector_palabra = X_docs[idx_palabra].toarray()
cos_term_sim = cosine_similarity(vector_palabra, X_docs)

# Se traen los índices de las palabras más similares de acuerdo al orden de distancia coseno
idx_similares = np.argsort(cos_term_sim[0])[::-1][1:6]
print(f"Indices de palabras similares: {idx_similares} \n")
print([(terms[i], np.round(cos_term_sim[0][i],4)) for i in idx_similares])

Indices de palabras similares: [ 546   64 2375 2556  431] 

[('comp', 0.2288), ('3d', 0.1686), ('svga', 0.1216), ('vga', 0.1199), ('card', 0.119)]


Todas las palabras que el código devuelve como más similares, hacen referencia de una u otra forma a temas gráficos en herramientas computacionales (`3d`,`card`) y/o electrónicas (`svga`,`vga`), por lo que parece identificar bien la relación.

Revisión palabra objetivo: **'games'**

In [498]:
# Índice de la palabra en `terms` fila correspondiente en la matriz de documentos
idx_palabra = terms.tolist().index(palabras_target[2])
vector_palabra = X_docs[idx_palabra].toarray()
cos_term_sim = cosine_similarity(vector_palabra, X_docs)

# Se traen los índices de las palabras más similares de acuerdo al orden de distancia coseno
idx_similares = np.argsort(cos_term_sim[0])[::-1][1:6]
print(f"Indices de palabras similares: {idx_similares} \n")
print([(terms[i], np.round(cos_term_sim[0][i],4)) for i in idx_similares])

Indices de palabras similares: [1056 2126 2118 2398 1797] 

[('game', 0.2081), ('season', 0.1802), ('scoring', 0.1686), ('team', 0.1648), ('played', 0.1495)]


En este caso las palabras similares, hacen referencia a temas relacionados con juegos sin especificar ninguno en particular, devuelve elementos genéricos de todo juego como pueden ser las temporadas(`season`), puntaje (`scoring`), equipo (`team`) o el acto de jugar en sí.

Revisión palabra objetivo: **'market'**

In [499]:
# Índice de la palabra en `terms` fila correspondiente en la matriz de documentos
idx_palabra = terms.tolist().index(palabras_target[3])
vector_palabra = X_docs[idx_palabra].toarray()
cos_term_sim = cosine_similarity(vector_palabra, X_docs)

# Se traen los índices de las palabras más similares de acuerdo al orden de distancia coseno
idx_similares = np.argsort(cos_term_sim[0])[::-1][1:6]
print(f"Indices de palabras similares: {idx_similares} \n")
print([(terms[i], np.round(cos_term_sim[0][i],4)) for i in idx_similares])

Indices de palabras similares: [1858  411  815  514  472] 

[('price', 0.1106), ('buy', 0.0936), ('economic', 0.0919), ('clipper', 0.0893), ('cheap', 0.0891)]


En este caso, la palabra objetivo `market`, devuelve como más similares, palabras relacionadas al ámbito de (super)mercado, como puede ser: `buy`, `price`, `cheap`, `economic`. En el caso de `clipper`, no se entiende bien la referencia, quizás por el idioma.

Revisión palabra objetivo: **'data'**

In [500]:
# Índice de la palabra en `terms` fila correspondiente en la matriz de documentos
idx_palabra = terms.tolist().index(palabras_target[4])
vector_palabra = X_docs[idx_palabra].toarray()
cos_term_sim = cosine_similarity(vector_palabra, X_docs)

# Se traen los índices de las palabras más similares de acuerdo al orden de distancia coseno
idx_similares = np.argsort(cos_term_sim[0])[::-1][1:6]
print(f"Indices de palabras similares: {idx_similares} \n")
print([(terms[i], np.round(cos_term_sim[0][i],4)) for i in idx_similares])

Indices de palabras similares: [2466  761  836 2319  837] 

[('transfer', 0.1574), ('disk', 0.1162), ('encrypted', 0.1161), ('stored', 0.1097), ('encryption', 0.1055)]


En este caso, se ve que la relación entre palabra objetivo y similares, hace referencia a todo lo relacionado con lo que se puede hacer con los datos (`transfer`, `encryption`, `stored`) o contenedores de los mismos (`disk`).